# Linear Mixed Models

The `lmer()` class makes it easy to work with linear-mixed models (multi-level models) estimated via maximum-likelihood or restricted-maximum-likelihood estimation

Random-effects can be specified using the same [formula syntax as lme4 in R](../pages/faqs.ipynb) and all methods are the same as `lm` models.


**Core Methods and Attributes**  

| Method | Description |
|--------|-------------|
| [`.fit()`](../api/models/lmer.md#pymer4.models.lmer.lmer.fit) | Fit a model using `lmer()` & `lmerTest()` |
| [`.anova()`](../api/models/lmer.md#pymer4.models.lmer.lmer.anova) | Calculate a Type-III ANOVA table using `joint_tests()` in R |
| [`.summary()`](../api/models/lmer.md#pymer4.models.base.model.summary) | Get a nicely formatted table containing `.result_fit` |
| [`.empredict()`](../api/models/lmer.md#pymer4.models.base.model.empredict) | Compute marginal predictions are arbitrary values of predictors |
| [`.emmeans()`](../api/models/lmer.md#pymer4.models.lmer.lmer.emmeans) | Compute marginal means/contrasts as specific factor levels |
| [`.simulate()`](../api/models/lmer.md#pymer4.models.lmer.lmer.simulate) | Simulate new observations from the fitted model |
| [`.predict()`](../api/models/lmer.md#pymer4.models.lmer.lmer.predict) | Make predictions given a new dataset |

`lmer` models store the results of their last estimation as `polars` Dataframes like `lm` models, but with a few more attributes:

| Attribute | Stores |
|--------|-------------|
| `.params`| parameter estimates |
| `.ranef`| cluster-level deviances for random-effects terms |
| `.ranef_var`| variances and co-variance for random-effects terms |
| `.fixef`| cluster-level estimates for random-effects terms (`.params + .ranef`); also commonly called [BLUPs](https://en.wikipedia.org/wiki/Best_linear_unbiased_prediction) |
| `.result_fit`| parameter estimates, errors, & inferential statistics |
| `.result_fit_stats`| model fit statistics |
| `.result_anova`| ANOVA table |


In [4]:
from pymer4.models import lmer
from pymer4 import load_dataset
sleep = load_dataset('sleep')

## Random Intercepts

In [6]:
model_i = lmer('Reaction ~ Days + (1 | Subject)', data=sleep)
model_i.fit()

As with `lm()` models, we can inspect estimates with the `.params` attribute. In the current model these are the estimates for the overall model intercept and slope for $Days$

In [3]:
model_i.params

term,estimate
str,f64
"""(Intercept)""",251.405105
"""Days""",10.467286


We can see estimates for each cluster (each $Subject$), commonly called BLUPs, in the `.fixef` attribute  

Because we only estimated a random *intercept* per $Subject$ all estimates (slopes) for $Days$ are the same

In [4]:
model_i.fixef.head()

level,(Intercept),Days
str,f64,f64
"""308""",292.188815,10.467286
"""309""",173.555551,10.467286
"""310""",188.296537,10.467286
"""330""",255.811547,10.467286
"""331""",261.621294,10.467286


These estimates are derived from the random deviances of each $Subject$ around the fixed parameter estimates. We can see those with `.ranef` which only contains a column for random intercepts

In [5]:
model_i.ranef.head()

level,(Intercept)
str,f64
"""308""",40.78371
"""309""",-77.849554
"""310""",-63.108567
"""330""",4.406442
"""331""",10.216189


We can see the variability across $Subject$'s using `.ranef_var`

In [6]:
model_i.ranef_var

group,term,estimate,conf_low,conf_high
str,str,f64,f64,f64
"""Subject""","""sd__(Intercept)""",37.123827,null,null
"""Residual""","""sd__Observation""",30.991234,null,null


Of course it's often easiest to see all this information at once using `.summary()`

In [7]:
model_i.summary()

GT(_tbl_data=shape: (6, 10)
┌─────────────┬─────────────┬────────────┬───────────┬───┬───────────┬───────────┬─────────┬───────┐
│ rfx         ┆ param       ┆ estimate   ┆ std_error ┆ … ┆ t_stat    ┆ df        ┆ p_value ┆ stars │
│ ---         ┆ ---         ┆ ---        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---     ┆ ---   │
│ str         ┆ str         ┆ f64        ┆ f64       ┆   ┆ f64       ┆ f64       ┆ str     ┆ str   │
╞═════════════╪═════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═════════╪═══════╡
│ Subject-sd  ┆ (Intercept) ┆ 37.123827  ┆ null      ┆ … ┆ null      ┆ null      ┆ null    ┆ null  │
│ Residual-sd ┆ Observation ┆ 30.991234  ┆ null      ┆ … ┆ null      ┆ null      ┆ null    ┆ null  │
│ null        ┆ null        ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null    ┆ null  │
│ Fixed       ┆ null        ┆ null       ┆ null      ┆ … ┆ null      ┆ null      ┆ null    ┆ null  │
│ Effects:    ┆             ┆            ┆           ┆   ┆           ┆           ┆         ┆       │
│ null        ┆ (Intercept) ┆ 251.405105 ┆ 9.746716  ┆ … ┆ 25.793826 ┆ 22.810199 ┆ <.001   ┆ ***   │
│ null        ┆ Days        ┆ 10.467286  ┆ 0.804221  ┆ … ┆ 13.015428 ┆ 161.0     ┆ <.001   ┆ ***   │
└─────────────┴─────────────┴────────────┴───────────┴───┴───────────┴───────────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x157f0a7e0>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='t_stat', type=<ColInfoTypeEnum.default: 1>, column_label='T-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='DF', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x174b0bb90>, _spanners=Spanners([]), _heading=Heading(title='Formula: lmer(Reaction~Days+(1|Subject))', subtitle=Md(text='Number of observations: *180*  \n            Confidence intervals: *parametric*  \n            ---------------------  \n            Log-likelihood: *-893*  \n            AIC: *1794* | BIC: *1807*  \n            Residual error: *30.991*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None), grpname=None, colname='param', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None), grpname=None, colname='param', rownum=2, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=Non

## Random Slopes

We can easily extend our model with random slopes by adding them to the formula. Before inspecting the parameter estimates it can be helpful to test whether we really *need* the extra parameters by comparing this model to our first model using a likelihood ratio test.

In [8]:
from pymer4.models import compare

model_s = lmer('Reaction ~ Days + (Days | Subject)', data=sleep)
compare(model_i, model_s)

GT(_tbl_data=shape: (2, 10)
┌─────────────┬─────────────┬─────────────┬──────┬───┬─────┬────────────┬───────┬─────┐
│ AIC         ┆ BIC         ┆ logLik      ┆ npar ┆ … ┆ Df  ┆ Pr(>Chisq) ┆ stars ┆ ids │
│ ---         ┆ ---         ┆ ---         ┆ ---  ┆   ┆ --- ┆ ---        ┆ ---   ┆ --- │
│ f64         ┆ f64         ┆ f64         ┆ i64  ┆   ┆ f64 ┆ str        ┆ str   ┆ i64 │
╞═════════════╪═════════════╪═════════════╪══════╪═══╪═════╪════════════╪═══════╪═════╡
│ 1794.465085 ┆ 1807.236913 ┆ -893.232543 ┆ 4    ┆ … ┆ 0.0 ┆            ┆       ┆ 1   │
│ 1755.628272 ┆ 1774.786013 ┆ -871.814136 ┆ 6    ┆ … ┆ 2.0 ┆ <.001      ┆ ***   ┆ 2   │
└─────────────┴─────────────┴─────────────┴──────┴───┴─────┴────────────┴───────┴─────┘, _body=<great_tables._gt_data.Body object at 0x174ae8e30>, _boxhead=Boxhead([ColInfo(var='ids', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='right', column_width=None), ColInfo(var='AIC', type=<ColInfoTypeEnum.default: 1>, column_label='AIC', column_align='right', column_width=None), ColInfo(var='BIC', type=<ColInfoTypeEnum.default: 1>, column_label='BIC', column_align='right', column_width=None), ColInfo(var='logLik', type=<ColInfoTypeEnum.default: 1>, column_label='logLik', column_align='right', column_width=None), ColInfo(var='npar', type=<ColInfoTypeEnum.default: 1>, column_label='npar', column_align='right', column_width=None), ColInfo(var='-2*log(L)', type=<ColInfoTypeEnum.default: 1>, column_label='-2*log(L)', column_align='right', column_width=None), ColInfo(var='Chisq', type=<ColInfoTypeEnum.default: 1>, column_label='Chisq', column_align='right', column_width=None), ColInfo(var='Df', type=<ColInfoTypeEnum.default: 1>, column_label='Df', column_align='right', column_width=None), ColInfo(var='Pr(>Chisq)', type=<ColInfoTypeEnum.default: 1>, column_label='Pr(>Chisq)', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x159848590>, _spanners=Spanners([]), _heading=Heading(title='Analysis of Deviance Table', subtitle=Md(text='Model 1: lmer(Reaction~Days+(1|Subject))   \nModel 2: lmer(Reaction~Days+(Days|Subject))   \n'), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x174b08650>, _formats=[<great_tables._gt_data.FormatInfo object at 0x174aea270>, <great_tables._gt_data.FormatInfo object at 0x174ae9130>], _substitutions=[<great_tables._gt_data.FormatInfo object at 0x174ae8e00>], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='

It looks it's worth it! 

Let's calculate bootstrapped confidence intervals, which will give us some uncertainty estimates around *both* fixed and random effects terms. By default we don't get confidence-intervals around random effects terms, but we can with bootstrapping.

We can see that our estimate of the standard-deviation across $Subject$'s intercepts is about ~25 with 95% CI of [11 36], while the standard-deviation of their slopes for $Days$ is about 6 with 95% CI of [3, 9]

In [10]:
model_s.fit(conf_method='boot', summary=True)

GT(_tbl_data=shape: (8, 10)
┌─────────────┬─────────────┬────────────┬────────────┬───┬──────────┬───────────┬─────────┬───────┐
│ rfx         ┆ param       ┆ estimate   ┆ conf_low   ┆ … ┆ t_stat   ┆ df        ┆ p_value ┆ stars │
│ ---         ┆ ---         ┆ ---        ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---     ┆ ---   │
│ str         ┆ str         ┆ f64        ┆ f64        ┆   ┆ f64      ┆ f64       ┆ str     ┆ str   │
╞═════════════╪═════════════╪════════════╪════════════╪═══╪══════════╪═══════════╪═════════╪═══════╡
│ Subject-sd  ┆ (Intercept) ┆ 24.740658  ┆ 12.039791  ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ Subject-sd  ┆ Days        ┆ 5.922138   ┆ 3.48035    ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ Subject-cor ┆ (Intercept) ┆ 0.065551   ┆ -0.489232  ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ Residual-sd ┆ Observation ┆ 25.591796  ┆ 22.596828  ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ null        ┆ null        ┆ null       ┆ null       ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ Fixed       ┆ null        ┆ null       ┆ null       ┆ … ┆ null     ┆ null      ┆ null    ┆ null  │
│ Effects:    ┆             ┆            ┆            ┆   ┆          ┆           ┆         ┆       │
│ null        ┆ (Intercept) ┆ 251.405105 ┆ 237.454598 ┆ … ┆ 36.83809 ┆ 16.999726 ┆ <.001   ┆ ***   │
│ null        ┆ Days        ┆ 10.467286  ┆ 7.334756   ┆ … ┆ 6.771481 ┆ 16.999984 ┆ <.001   ┆ ***   │
└─────────────┴─────────────┴────────────┴────────────┴───┴──────────┴───────────┴─────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x304225370>, _boxhead=Boxhead([ColInfo(var='rfx', type=<ColInfoTypeEnum.default: 1>, column_label='Random Effects:', column_align='left', column_width=None), ColInfo(var='param', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='estimate', type=<ColInfoTypeEnum.default: 1>, column_label='Estimate', column_align='right', column_width=None), ColInfo(var='conf_low', type=<ColInfoTypeEnum.default: 1>, column_label='CI-low', column_align='right', column_width=None), ColInfo(var='conf_high', type=<ColInfoTypeEnum.default: 1>, column_label='CI-high', column_align='right', column_width=None), ColInfo(var='std_error', type=<ColInfoTypeEnum.default: 1>, column_label='SE', column_align='right', column_width=None), ColInfo(var='t_stat', type=<ColInfoTypeEnum.default: 1>, column_label='T-stat', column_align='right', column_width=None), ColInfo(var='df', type=<ColInfoTypeEnum.default: 1>, column_label='DF', column_align='right', column_width=None), ColInfo(var='p_value', type=<ColInfoTypeEnum.default: 1>, column_label='p', column_align='left', column_width=None), ColInfo(var='stars', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x30421c140>, _spanners=Spanners([]), _heading=Heading(title='Formula: lmer(Reaction~Days+(Days|Subject))', subtitle=Md(text='Number of observations: *180*  \n            Confidence intervals: *boot*  \n            Bootstrap Iterations: *1000*  \n            ---------------------  \n            Log-likelihood: *-871*  \n            AIC: *1755* | BIC: *1774*  \n            Residual error: *25.592*  \n        '), preheader=None), _stubhead=None, _source_notes=[Md(text='Signif. codes: *0 *** 0.001 ** 0.01 * 0.05 . 0.1*')], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=['param'], rows=None), grpname=None, colname='param', rownum=0, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), StyleInfo(locname=LocBody(columns=['param'], rows=None), grpname=None, colname='param', rownum=1, colnum=None, styles=[CellStyleText(color=None, font=None, size=None, align=None, v_align=None, style='italic', weight=None, stretch=None, decorate=None, transform=None, whitespace=None)]), Style

## Model issues

When fitting more complex models or models with redundant parameters you'll often run into model *convergence* issues and/or model *singularity* issues.

Models will print out warning messages from R when this happens to let you know. But you can also suppress these by passing `verbose = False` to `.fit()`

In [2]:
bad_model = lmer('Reaction ~ Days + (Days | Subject) + (1 | Days)', data=sleep)
bad_model.fit()

R messages: 
boundary (singular) fit: see help('isSingular')

R messages: 
boundary (singular) fit: see help('isSingular')



In [3]:
# No warning!
bad_model.fit(verbose=False)

You can always see if there were any suppressed warnings using `.show_logs()`

In [4]:
bad_model.show_logs()

R messages: 
boundary (singular) fit: see help('isSingular')



Convergence issues can often be due to a lack of variance for some of the random-effects terms. The "gradient" value should be a very very tiny number that reflects the change is the optimizer's error during fitting. Values close to or > 0 suggest the optimizer struggled to find the "best" parameter estimates.

In [6]:
mtcars = load_dataset('mtcars')

bad_model = lmer('mpg ~ 1 + (hp | cyl)', data=mtcars)
bad_model.set_factors('cyl')
bad_model.fit()

R messages: 
Convergence status
: [1] FALSE
attr(,"gradient")
[1] 1.145261



We can see that the variability around the random slope across $hp$ for $cyl$ is very small (~0.034) which makes this challenging for the optimizer

In [7]:
bad_model.ranef_var

group,term,estimate,conf_low,conf_high
str,str,f64,f64,f64
"""cyl""","""sd__(Intercept)""",6.192348,null,null
"""cyl""","""cor__(Intercept).hp""",-0.869679,null,null
"""cyl""","""sd__hp""",0.034038,null,null
"""Residual""","""sd__Observation""",3.120614,null,null


We can attempt to bootstrap the model to see how much the variability changes. We again see covergence warnings and that the range of estimates includes very small values for the random slope.

We're also having hard time estimating the correlation between intercept and slope leading to a huge confidence interval

In [8]:
bad_model.fit(conf_method='boot')
bad_model.ranef_var

R messages: 
Convergence status
: [1] FALSE
attr(,"gradient")
[1] 1.145261



group,term,estimate,conf_low,conf_high
str,str,f64,f64,f64
"""cyl""","""sd__(Intercept)""",6.192348,0.0,13.387067
"""cyl""","""cor__(Intercept).hp""",-0.869679,-1.0,1.0
"""cyl""","""sd__hp""",0.034038,3.3944e-7,0.087685
"""Residual""","""sd__Observation""",3.120614,2.287559,3.949821


We can try to simplify the model by remove the correlation between random-effects terms. The model still fails to convergence, but the gradient is much smaller now.

It looks like the lack of variability in the random slope is still giving the optimizer issues:

In [9]:
simple_model = lmer('mpg ~ 1 + (1 | cyl) + (0 + hp | cyl)', data=mtcars)
simple_model.set_factors('cyl')
simple_model.fit(conf_method='boot')
simple_model.ranef_var

R messages: 
Convergence status
: [1] FALSE
attr(,"gradient")
[1] 0.1034371



group,term,estimate,conf_low,conf_high
str,str,f64,f64,f64
"""cyl""","""sd__(Intercept)""",5.706231,0.0,11.294004
"""cyl.1""","""sd__hp""",0.012059,0.0,0.04829
"""Residual""","""sd__Observation""",3.182668,2.266804,4.016254


Let's simplify further by retaining only random-intercepts:

In [10]:
simpler_model = lmer('mpg ~ 1 + (1 | cyl)', data=mtcars)
simpler_model.set_factors('cyl')
simpler_model.fit(conf_method='boot')
simpler_model.ranef_var

group,term,estimate,conf_low,conf_high
str,str,f64,f64,f64
"""cyl""","""sd__(Intercept)""",5.761425,0.392901,11.721516
"""Residual""","""sd__Observation""",3.222468,2.349387,4.015793


No more warnings! We can verify by checking out the logs:

In [ ]:
# Empty
simpler_model.show_logs()

And convergence status, which shows successful convergence and a tiny gradient:

In [15]:
print(simpler_model.convergence_status)

Convergence status
: [1] TRUE
attr(,"gradient")
[1] 7.001001e-06



## Summary

`lmer()` models support all the same features as `lm()` models for working with categorical predictors and generating marginal estimates and comparisons. 

So be sure to check out the [linear regression](../tutorials/01_lm.ipynb) and [ANOVA](../tutorials/02_categorical.ipynb) tutorials for more details.